In [1]:
import os
import json
import dimensions_search_api_client as dscli
import RichContextAPI
import pandas as pd

In [2]:
datasets_path = "/Users/sophierand/RCDatasets/datasets.json"
with open(datasets_path) as json_file:
    datasets = json.load(json_file)

In [33]:
def flatten(l):
    fl = [item for sublist in l for item in sublist]
    return fl

def dimensions_run_exact_string_search(string,api_client):
    search_string = 'search publications in full_data for "\\"{}\\"" return publications[doi+title+journal+author_affiliations]'.format(string)
    api_response = api_client.execute_query(query_string_IN = search_string )
    return api_response

def gen_dimensions_linkages(ds_id, search_terms,api_client):
    md_list = []
    author_values = []
    for i in search_terms:
        dimension_return = run_exact_string_search(string = i,api_client = api_client)
        if dimensions_return is not None or 'publications' in dimensions_return.keys():
            dimension_return_pub = dimension_return['publications']
            ds_pub_list = []
            for d in dimension_return_pub:
                try:
                    md_dict = {'title':d['title'],'journal':d['journal']['title'],'search_string':i,'doi':d['doi'],'datasets':ds_id}
                    if 'author_affiliations' in d.keys():
                        author_list = d['author_affiliations'][0]
                        for a in author_list:
                            auth_dict = {'first_name':a['first_name'],'last_name':a['last_name'],'affiliation':a['affiliations'][0]['name']}
                            author_values.append(auth_dict)
                        md_dict.update({'authors':author_values})
                    elif 'author_affiliations' not in d.keys():
                        pass
                    ds_pub_list.append(md_dict)
                except:
                    pass
            md_list.append(ds_pub_list)
        elif dimensions_return is None or 'publications' not in dimension_return.keys():
            pass
    md_list = flatten(md_list)
    return md_list


def export_dimensions_csv(md_list,file_name):
    keys = ['title','doi','journal','search_string','datasets']
    md_list_csv = [{k:m[k] for k in keys} for m in md_list]
    md_df_csv = pd.DataFrame.from_dict(md_list_csv)
    md_df_csv = md_df_csv.drop_duplicates()
    md_df_csv.to_csv(file_name,index = False)

#### Connect to Dimensions

In [4]:
api_client = RichContextAPI.connect_dimensions_api()

#### SED

In [5]:
sed_datasets = [{'id':d['id'],'search_terms':[d['title']] + d['alt_title']} for d in datasets if d['id'] == 'dataset-370']
sed_datasets

[{'id': 'dataset-370',
  'search_terms': ['Survey of Earned Doctorates', 'SED', 'SESTAT']}]

In [6]:
sed_ds_id = sed_datasets[0]['id']
sed_search_terms = sed_datasets[0]['search_terms']

In [7]:
sed_md_list = gen_dimensions_linkages(ds_id = sed_ds_id, search_terms = sed_search_terms,api_client = api_client)

In [10]:
export_dimensions_csv(md_list = sed_md_list,file_name = 'sed_dimensions.csv')

#### SDR

In [11]:
sdr_datasets = [{'id':d['id'],'search_terms':[d['title']] + d['alt_title']} for d in datasets if d['id'] == 'dataset-371']
sdr_datasets

[{'id': 'dataset-371',
  'search_terms': ['Survey of Doctorate Recipients', 'SDR']}]

In [17]:
sdr_ds_id = sdr_datasets[0]['id']
sdr_search_terms = sdr_datasets[0]['search_terms']
sdr_search_terms
sdr_ds_id

'dataset-371'

In [14]:
sdr_md_list = gen_dimensions_linkages(ds_id = sdr_ds_id, search_terms = sdr_search_terms,api_client = api_client)

In [18]:
export_dimensions_csv(md_list = sdr_md_list,file_name = 'sdr_dimensions.csv')

#### UMETRICS

In [20]:
um_search_terms = ['UMETRICS','Universities: Measuring the Impacts of Research on Innovation, Competitiveness, and Science']

In [32]:
i = um_search_terms[1]
dimension_return = run_exact_string_search(string = i,api_client = api_client)


Non-JSON response:
<!DOCTYPE html>
<html lang="en">

<head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">

    <meta http-equiv="X-UA-Compatible" content="IE=EDGE">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>Dimensions</title>
    <link rel="icon" type="image/png" href="/files/favicon-16x16.png"
        sizes="16x16">

    <style>
        /* latin */
        @font-face {
            font-family: 'Roboto';
            font-style: normal;
            font-weight: 400;
            src: local('Roboto'), local('Roboto-Regular'), url(https://fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxKKTU1Kg.woff2) format('woff2');
                unicode-range: U+0000-00FF, U+0131, U+0152-0153, U+02BB-02BC, U+02C6, U+02DA, U+02DC, U+2000-206F, U+2074, U+20AC, U+2122, U+2191, U+2193, U+2212, U+2215, U+FEFF, U+FFFD;
        }

        /* latin */
        @font-face {
            font-family: 'Roboto';
            font-style: normal;
   

In [31]:
dimension_return

{'_stats': {'total_count': 16690},
 '_warnings': ["Field 'author_affiliations' is deprecated in favor of authors. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details"],
 'publications': [{'doi': '10.1166/jnn.2020.17216',
   'journal': {'id': 'jour.1297328',
    'title': 'Journal of Nanoscience and Nanotechnology'},
   'title': 'pH-Responsively Water-Retaining Controlled-Release Fertilizer Using Humic Acid Hydrogel and Nano-Silica Aqueous Dispersion',
   'author_affiliations': [[{'first_name': 'Yuhua',
      'last_name': 'Niu',
      'corresponding': '',
      'orcid': '',
      'current_organization_id': '',
      'researcher_id': '',
      'affiliations': [{'id': 'grid.454711.2',
        'name': 'Shaanxi University of Science and Technology',
        'city': "Xi'an",
        'city_id': 1790630,
        'country': 'China',
        'country_code': 'CN',
        'state': None,
        'state_code': None}]},
     {'first_name': 'Ruyuan',
      'last_name': 'K

In [29]:
um_md_list = gen_dimensions_linkages(ds_id = 'tbd', search_terms = um_search_terms,api_client = api_client)

NameError: name 'dimensions_return' is not defined

In [23]:
dimension_return

NameError: name 'dimension_return' is not defined

Scratch

In [ ]:
file_name = 'SED_dimensions_linkages.json'

In [ ]:
with open(file_name, 'w') as outfile:
    json.dump(md_list, outfile, indent = 2)

In [ ]:
keys = ['title','doi','journal','search_string','datasets']

md_list_csv = [{k:m[k] for k in keys} for m in md_list]